In [1]:
import numpy as np
import cv2
import mediapipe as mp
from main import HeadPoseEstimator
from utils import DataCleaner, split_train_dev_x_y, draw_axis

In [2]:
np.random.seed(27)

cleaner = DataCleaner(
            '/home/ahmad/projects/python/head_pose_estimation/AFLW2000-3D/AFLW2000')
df = cleaner.get_cleaned_df()
X_train, Y_train, X_dev, Y_dev = split_train_dev_x_y(df, train_size=0.8)

model = HeadPoseEstimator()
model = model.fit(X_train, Y_train)
# model.report(X_train, Y_train, X_dev, Y_dev)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


variance ratio = 0.9971236094417085


In [3]:
# cap = cv2.VideoCapture('/home/ahmad/Documents/da7eeh.mp4')
# # Obtain frame size information using get() method
# frame_width = int(cap.get(3))
# frame_height = int(cap.get(4))
# frame_size = (frame_width,frame_height)
# fps = 20
# output = cv2.VideoWriter(
#     '/home/ahmad/Documents/da7eeh_mod.mp4', 
#     cv2.VideoWriter_fourcc(*'XVID'), 
#     20, 
#     frame_size) 
    
# # Loop until the end of the video
# # i = 0
# while cap.isOpened() : # and i < 150:
#     # i += 1
#     success, image = cap.read()
#     if success:
#         faceModule = mp.solutions.face_mesh
#         with faceModule.FaceMesh(static_image_mode=True) as faces:
#             # loading the image
#             results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#             if results.multi_face_landmarks: 
#                 # looping over the faces in the image
#                 # many_faces_image_paths.append(index)
#                 # print(image_path, len(results.multi_face_landmarks))
#                 img_h, img_w, img_c = image.shape
#                 for face in results.multi_face_landmarks:
#                     features = np.array([
#                         [
#                             int(lm.x * img_w),
#                             int(lm.y * img_h)
#                         ]
#                         for lm in face.landmark 
#                     ], dtype=np.float64)
#                     nose = features[1]    
#                     features = cleaner.get_features(features)
#                     # print('features shape =', features.shape)
#                     features = cleaner.transform(features.reshape(1, -1))
#                     pitch, yaw, roll = model.predict(features)
#                     draw_axis(image, pitch, yaw, roll, nose[0], nose[1])
                
#                 output.write(image)
 
#         key = 0xFF & cv2.waitKey(1)
#         if key & 0xFF == 27: #Exit program when the user presses 'esc'
#             break
#     else:
#         break
# cv2.destroyAllWindows()
# cap.release()
# output.release()


In [4]:
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width,frame_height)
fps = 20
output = cv2.VideoWriter(
    '/home/ahmad/Documents/my_video.mp4', 
    cv2.VideoWriter_fourcc(*'XVID'), 
    20, 
    frame_size)
while cap.isOpened():
    success, image = cap.read()
    if success:
        faceModule = mp.solutions.face_mesh
        with faceModule.FaceMesh(static_image_mode=True) as faces:
            # loading the image
            results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if results.multi_face_landmarks: 
                # looping over the faces in the image
                # many_faces_image_paths.append(index)
                # print(image_path, len(results.multi_face_landmarks))
                img_h, img_w, img_c = image.shape
                for face in results.multi_face_landmarks:
                    features = np.array([
                        [
                            int(lm.x * img_w),
                            int(lm.y * img_h)
                        ]
                        for lm in face.landmark 
                    ], dtype=np.float64)
                    nose = features[1].reshape(1, -1)    
                    features = cleaner.get_features(features)
                    features = cleaner.transform(features.reshape(1, -1))
                    pitch, yaw, roll = model.predict(features)
                    draw_axis(image, yaw, pitch, roll, nose[0,0], nose[0,1])
                
                cv2.imshow('HPE', image)
                output.write(image)

        key = 0xFF & cv2.waitKey(1)
        if key & 0xFF == 27: #Exit program when the user presses 'esc'
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()
output.release()


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
